In [4]:
import pandas as pd

In [5]:
# census raw data
file_path = "data/census-tract-data.csv"  # Update this if needed
df = pd.read_csv(file_path)

print("Original Columns:", df.columns.tolist())
df.columns = df.columns.str.lower()


# rename columns
column_mapping = {
    "geoid": "Census_Tract_ID",
    "geocode": "Tract_Code",
    "tract": "Tract_Number",
    "p0020001": "Total_Population",
    "p0020002": "White_Population",
    "p0020005": "Black_Population",
    "p0020006": "Native_Population",
    "p0020007": "Asian_Population",
    "p0020008": "Pacific_Population",
    "p0020009": "Other_Race_Population",
    "p0020010": "Two_or_More_Races",
    "p0040002": "Hispanic_Population",
    "h0010001": "Total_Housing_Units",
    "h0010002": "Occupied_Housing_Units",
    "h0010003": "Vacant_Housing_Units"
}

# check for actual existing columns
existing_columns = {col: new_col for col, new_col in column_mapping.items() if col in df.columns}
df = df.rename(columns=existing_columns)

print("Renamed Columns:", df.columns.tolist())

# drop unecessary columns
drop_columns = ["fileid", "stusab", "sumlev", "region", "division", "state", "county", "cousub"]
df = df.drop(columns=[col for col in drop_columns if col in df.columns], errors="ignore")

# convert numeric columns to useful names
numeric_cols = list(existing_columns.values())
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors="coerce")

# handle missing data columns
if "Total_Population" in df.columns:
    df = df.dropna(subset=["Total_Population"])
else:
    print("'Total_Population' column not found.")

# calculate demographic percentages
race_columns = []
if "White_Population" in df.columns:
    df["Percent_White"] = (df["White_Population"] / df["Total_Population"]) * 100
    race_columns.append("Percent_White")
if "Black_Population" in df.columns:
    df["Percent_Black"] = (df["Black_Population"] / df["Total_Population"]) * 100
    race_columns.append("Percent_Black")
if "Hispanic_Population" in df.columns:
    df["Percent_Hispanic"] = (df["Hispanic_Population"] / df["Total_Population"]) * 100
    race_columns.append("Percent_Hispanic")
if "Asian_Population" in df.columns:
    df["Percent_Asian"] = (df["Asian_Population"] / df["Total_Population"]) * 100
    race_columns.append("Percent_Asian")
if all(x in df.columns for x in ["Native_Population", "Pacific_Population", "Other_Race_Population", "Two_or_More_Races"]):
    df["Percent_Other"] = ((df["Native_Population"] + df["Pacific_Population"] +
                            df["Other_Race_Population"] + df["Two_or_More_Races"]) / df["Total_Population"]) * 100
    race_columns.append("Percent_Other")

df.fillna(0, inplace=True)

# normalize percentages and remove extremely low populations
if len(race_columns) > 0:
    df["Total_Percent"] = df[race_columns].sum(axis=1)
    df.loc[df["Total_Percent"] > 100, race_columns] = df.loc[df["Total_Percent"] > 100, race_columns].div(df["Total_Percent"], axis=0) * 100
    df = df.drop(columns=["Total_Percent"])

# 
df = df[df["Total_Population"] >= 10]

df[race_columns] = df[race_columns].round(2)

# save cleaned data
output_file = "cleaned_2020_census_data.csv"
df.to_csv(output_file, index=False)

print(f"Cleaned census data saved as '{output_file}'.")

Original Columns: ['FILEID', 'STUSAB', 'SUMLEV', 'GEOID', 'GEOCODE', 'REGION', 'DIVISION', 'STATE', 'COUNTY', 'COUSUB', 'TRACT', 'P0020001', 'P0020005', 'P0020006', 'P0020002', 'P0020008', 'P0020007', 'P0020009', 'P0020010', 'P0020011', 'P0040001', 'P0040005', 'P0040006', 'P0040002', 'P0040008', 'P0040007', 'P0040009', 'P0040010', 'P0040011', 'P0050001', 'P0050002', 'P0050003', 'P0050004', 'P0050005', 'P0050006', 'P0050007', 'P0050008', 'P0050009', 'P0050010', 'H0010001', 'H0010002', 'H0010003']
Renamed Columns: ['fileid', 'stusab', 'sumlev', 'Census_Tract_ID', 'Tract_Code', 'region', 'division', 'state', 'county', 'cousub', 'Tract_Number', 'Total_Population', 'Black_Population', 'Native_Population', 'White_Population', 'Pacific_Population', 'Asian_Population', 'Other_Race_Population', 'Two_or_More_Races', 'p0020011', 'p0040001', 'p0040005', 'p0040006', 'Hispanic_Population', 'p0040008', 'p0040007', 'p0040009', 'p0040010', 'p0040011', 'p0050001', 'p0050002', 'p0050003', 'p0050004', 'p0